In [1]:
using ChaosCrypto

LoadError: LoadError: ArgumentError: ChaosCrypto not found in path
while loading In[1], in expression starting on line 1

In [ ]:
#Funciones Runge Kutta
function runge2(f,x,t,dt)
    k1 = f(x,t)
    k2 = f(x+2dt*k1/3, t+2*dt/3)
    (k1/4+3k2/4)
end

function runge4(f,x,t,dt)
    k1 = f(x,t)
    k2 = f(x+dt*k1/2, t+dt/2)
    k3 = f(x+dt*k2/2,t+dt/2)
    k4 = f(x+dt*k3,t + dt)
    (k1+2k2+2k3+k4)/6
end

function superrunge(f,x,t,dt)
    x1x,u1x,x1y,u1y,x2x,u2x,x2y,u2y = x
    normi = ((x1x-x2x)^2+(x1y-x2y)^2)^(-3/2)
    if normi < 1e-3
        return "fin"
    else
        k1 = f(x,t)
        k2 = f(x+dt*k1/2, t+dt/2)
        k3 = f(x+dt*k2/2,t+dt/2)
        k4 = f(x+dt*k3,t + dt)
        (k1+2k2+2k3+k4)/6
    end
end

In [ ]:
#Integracion
function integrar(f, x0, t0, t_final, dt, metodo = 4)
    # necesito una function f(x,t)
    # que come un vector x, el tiempo t
    # y regresa un vector (el campo vectorial evaluado en x,t)
    
    tiempos = [t0]
    xs = typeof(x0)[x0]

    #x = copy(x0)  # ver pythontutor.com
    
    x = x0
    
    for t in t0:dt:t_final
        if metodo == 1
            k = f(x,t)
        elseif metodo == 2
            k = runge2(f,x,t,dt)
        elseif metodo == 4
            k = runge4(f,x,t,dt)
        elseif metodo == 20
            k = superrunge(f,x,t,dt)
            if k == "fin"
                return xs, tiempos
            end
        else
            error("Los únicos métodos son 1 2 ó 4")
        end
        x_nueva = x + dt*k
        
        push!(xs, x)
        push!(tiempos, t)
        
        x = x_nueva
        
    end
    
    xs, tiempos
end

In [ ]:
#Funcion para generar Taylor
function generarTaylor(condIni, funcion)
    funcion(condIni)
end

#Funcion para generar serie
function generarSerie(polTalor)
    x = Float64[]
    push!(x, polTalor.ini)
    valores = copy(polTalor.coef)
    for i in 1:length(valores)
        push!(x, valores[i]/i)
    end
    x
end

#Funcion para generar intervalos
function generaIntervalo(lista)
    p = length(lista)
    h = lista[end]
    while (lista[p] == 0)
        p -= 1
        h = lista[p]
    end
    ϵ = 1e-3
    (ϵ/h)^(1/p)
end

#Funcion horner
function horner(x,h = 1e-3)
    p = length(x)
    xt = zeros(x)
    xt[p-1] = x[p-1] + h*x[p]
    for i in 3:p
        xt[p-i+1] = x[p-i+1] + h*xt[p-i+1+1]
    end
    xt[1]
end

#Funcion Lorenz
function lorenz(xx, t)
    x, y, z, xr, yr, zr= xx
    AD.Taylor(x, [σ])*(AD.Taylor(x,[y])-AD.Taylor(x,[x])),
    AD.Taylor(y,[x])*(AD.Taylor(y,[ρ])-AD.Taylor(y,[z]))-AD.Taylor(y,[y]),
    AD.Taylor(z,[x])*AD.Taylor(z,[y])-AD.Taylor(z,[β])*AD.Taylor(z,[z]),
    AD.Taylor(xr,[σ])*(AD.Taylor(xr,[y])-AD.Taylor(xr,[xr])),
    AD.Taylor(yr,[ρ])*AD.Taylor(yr,[xr])-AD.Taylor(yr,[yr])-AD.Taylor(yr,[xr])*AD.Taylor(yr,[zr]),
    AD.Taylor(zr,[xr])*AD.Taylor(zr,[yr])-AD.Taylor(zr,[β])*AD.Taylor(zr,[zr])
end

In [ ]:
#Funcion integradora 
function integrador(x0, f, tf)
    a = generarTaylor(x0,f)
    b = map(generarSerie,a)
    suma = map(horner,b)
    sol = Array{Float64,1}[x0, [suma...]]
    t = [0.,1e-3]
    while t[end] < tf
        a = generarTaylor(sol[end],f)
        b = map(generarSerie,a)
        suma = map(horner,b)
        push!(sol,[suma...])
        push!(t,t[end]+1e-3)
    end
    sol,t
end

In [ ]:
σ = 10
ρ = 60
β = 8/3

In [ ]:
function lorenzSin(xx,t)
    x, y, z, xr, yr, zr = xx
    
    m = sin(t)
    s = x + m
    
    [σ*(y-x), (ρ*x - y - x*z), x*y-β*z,
     σ*(yr-xr), (ρ*s - yr - s*zr) , (s*yr - β*zr)]
end

In [ ]:
xsSin, tsSin = integrar(lorenzSin,[1., 1., 1., 1., 1., 1.],0. ,30. ,1e-3,4)

In [ ]:
m = sin(tsSin)
s = [x[1] for x in xsSin] + m
m_hat = s - [x[4] for x in xsSin]

In [ ]:
plot(tsSin, m)
plot(tsSin, m_hat)
legend()

In [ ]:
function paso(x)
    if x > 2 && x < 3
        return 1.
    end 
    0.
end

In [ ]:
function lorenzpaso(xx,t)
    x, y, z, xr, yr, zr = xx
    
    m = paso(t)
    s = x + m
    
    [σ*(y-x), (ρ*x - y - x*z), x*y-β*z,
     σ*(yr-xr), (ρ*s - yr - s*zr) , (s*yr - β*zr)]
end

In [ ]:
xsPaso, tsPaso = integrar(lorenzpaso,[1., 1., 1., 1., 10, 10],0. ,30. ,1e-3,4)

In [ ]:
mpaso = [paso(p) for p in tsPaso]

In [ ]:
s = [x[1] for x in xsPaso] + m
m_hat_paso = s - [x[4] for x in xsPaso]

In [ ]:
plot(tsPaso, mpaso)
plot(tsPaso, m_hat_paso)
legend(

In [ ]:
plot(tsPaso, mpaso)
plot(tsPaso, m_hat_paso)
legend()